<a href="https://colab.research.google.com/github/obeabi/NaturalLanguageProcessing/blob/master/TwitterSentiment_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment140 - A Twitter Sentiment Analysis Tool
# Written by Abiola Obembe
## Date: 2020-12-


Abstract
The data is a CSV with emoticons removed. Data file format has 6 fields:
0 - the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)
1 - the id of the tweet (2087)
2 - the date of the tweet (Sat May 16 23:58:44 UTC 2009)
3 - the query (lyx). If there is no query, then this value is NO_QUERY.
4 - the user that tweeted (robotickilldozr)
5 - the text of the tweet (Lyx is cool)



## Step 1: Import dependencies

In [31]:
#Import libraries
import numpy as np
import pandas as pd
import re
import math
from bs4 import BeautifulSoup
#from nltk.corpus import stopwords
#from nltk.stem.porter import PorterStemmer

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
#import tensorflow-datasets as tfds
from tensorflow.keras import layers
import tensorflow_datasets as tfds
print("The tensorflow version is :", tf.__version__)


print("Dependencies installed succesffuly!")


The tensorflow version is : 2.4.0
Dependencies installed succesffuly!


## Step 2: Data Preprocessing

In [32]:
# Load data files
cols = ["sentiment", "id", "date", "query", "user", "text"]

train_data = pd.read_csv("traindata.csv",error_bad_lines=False,
    header=None,names=cols,engine="python",encoding="latin1")

test_data = pd.read_csv(
    "testdata.csv",error_bad_lines=False,
    header=None, names=cols,engine="python",encoding="latin1")


print("Training set :", train_data.shape )
print("Test set :", test_data.shape)

train_data.head()

Training set : (1600000, 6)
Test set : (498, 6)


,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [33]:
# clean data
train_data.drop(['id','date', 'query','user'], axis = 1, inplace= True)
train_data.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [34]:
# Examine sentiment data labels
train_data['sentiment'].value_counts()

4    800000
0    800000
Name: sentiment, dtype: int64

In [35]:
# Function to clean
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    # Removing the @
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    # Removing the URL links
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    # Keeping only letters
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    # Removing additional whitespaces
    tweet = re.sub(r" +", ' ', tweet)
    tweet = tweet.lower()
    
    return tweet

In [36]:
# apply function on dataset
data_train_clean = [ clean_tweet(tweet) for tweet in train_data.text]


In [37]:
# Evaluate the sentiment column
set(train_data.sentiment.values)

{0, 4}

In [38]:
# Let's set the sentiment values labelled 4 as 1
data_labels = train_data.sentiment.values
data_labels[data_labels == 4] = 1
set(data_labels)

{0, 1}

In [39]:
data_labels

array([0, 0, 0, ..., 1, 1, 1])

In [42]:
# Tokenization method 1
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    data_train_clean, target_vocab_size=2**10)

data_inputs = [tokenizer.encode(sentence) for sentence in data_train_clean]

In [43]:
# Padding
MAX_LEN = max([len(sentence) for sentence in data_inputs])
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs,
                                                            value=0,
                                                            padding="post",
                                                            maxlen=MAX_LEN)

In [44]:
# Split dataset into test and train set (due to orderd nature of traget values 0 and 1)
test_idx = np.random.randint(0, 800000, 8000)
test_idx = np.concatenate((test_idx, test_idx+800000))
test_inputs = data_inputs[test_idx]
test_labels = data_labels[test_idx]
train_inputs = np.delete(data_inputs, test_idx, axis=0)  # deletes the test indices from train inputs
train_labels = np.delete(data_labels, test_idx)


## Step 3: Build Model

In [46]:
class DCNN(tf.keras.Model):
    
    def __init__(self,
                 vocab_size,
                 emb_dim=128,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="dcnn"):
        super(DCNN, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocab_size,emb_dim)

        self.bigram = layers.Conv1D(filters=nb_filters, kernel_size=2, padding="valid", activation="relu")
        
        self.trigram = layers.Conv1D(filters=nb_filters,kernel_size=3, padding="valid", activation="relu")
        
        self.fourgram = layers.Conv1D(filters=nb_filters, kernel_size=4, padding="valid", activation="relu")
        
        self.pool = layers.GlobalMaxPool1D() # no training variable so we can
                                             # use the same layer for each
                                             # pooling step
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1, activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes, activation="softmax")
    
    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

## Step 4: Training the Model

In [47]:
# Configuration details
VOCAB_SIZE = tokenizer.vocab_size

EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 512
NB_CLASSES = len(set(train_labels))

DROPOUT_RATE = 0.2

BATCH_SIZE = 32
NB_EPOCHS = 2

In [49]:
# Let's initiate the deep convolution neural network object
Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [50]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [51]:
# Train the model here
Dcnn.fit(train_inputs,
         train_labels,
         batch_size=BATCH_SIZE,
         epochs=NB_EPOCHS)
#ckpt_manager.save()

Epoch 1/2
17658/49503 [=========>....................] - ETA: 1:12:44 - loss: 0.4756 - accuracy: 0.7688

KeyboardInterrupt: ignored

## Step 6: Evaluate the model

In [ ]:
#results = Dcnn.evaluate(test_inputs, test_labels, batch_size=BATCH_SIZE)
results = Dcnn.evaluate(test_inputs, test_labels)
print(results)

In [ ]:
Dcnn(np.array([tokenizer.encode("bad teacher")]), training=False).numpy()

In [ ]:
# Summary of  trained model
Dcnn.summary()

In [ ]:
# Save the entire model as a SavedModel.
#!mkdir -p saved_model
Dcnn.save('saved_model/my_model_2epoch')

In [ ]:
# load model
new_model = tf.keras.models.load_model('saved_model/my_model_5epoch')

# Check its architecture
new_model.summary()

In [ ]:
loss, acc = new_model.evaluate(test_inputs, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_inputs).shape)